## RLVR Example - Finetuning with Sagemaker

This notebook demonstrates basic user flow for RLVR Finetuning from a model available in Sagemaker Jumpstart.
Information on available models on jumpstart: https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-latest.html

### Setup and Configuration

Initialize the environment by importing necessary libraries and configuring AWS credentials

In [1]:

from sagemaker.train.rlvr_trainer import RLVRTrainer
from sagemaker.train.configs import InputData
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage
from sagemaker.train.common import TrainingType


import boto3
from sagemaker.core.helper.session_helper import Session

# For MLFlow native metrics in Trainer wait, run below line with approriate region
# os.environ["SAGEMAKER_MLFLOW_CUSTOM_ENDPOINT"] = "https://mlflow.sagemaker.us-east-1.app.aws"



[12/02/25 17:54:14] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=600716;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=966746;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=25130;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=835202;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

### Create RLVRTrainer
**Required Parameters** 

* `model`: base_model id on Sagemaker Hubcontent that is available to finetune (or) ModelPackage artifacts

**Optional Parameters**
* `custom_reward_function`: Custom reward function/Evaluator ARN
* `model_package_group_name`: ModelPackage group name or ModelPackageGroup
* `mlflow_resource_arn`: MLFlow app ARN to track the training job
* `mlflow_experiment_name`: MLFlow app experiment name(str)
* `mlflow_run_name`: MLFlow app run name(str)
* `training_dataset`: Training Dataset - either Dataset ARN or S3 Path of the dataset (Please note these are required for a training job to run, can be either provided via Trainer or .train())
* `validation_dataset`: Validation Dataset - either Dataset ARN or S3 Path of the dataset
* `s3_output_path`: S3 path for the trained model artifacts

In [2]:
# For fine-tuning (prod)
rlvr_trainer = RLVRTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage] 
    model_package_group_name="sdk-test-finetuned-models", #"test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing/input_data/rlvr-rlaif-test-data/train_285.jsonl",     #"arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0", #Optional[]
    s3_output_path="s3://mc-flows-sdk-testing/output/",
    #sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    #role="arn:aws:iam::052150106756:role/Admin",
    accept_eula=True
)

[12/01/25 13:07:39] INFO     SageMaker session not provided. Using default Session.                  ]8;id=77955;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=989493;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

                    INFO     Runs on sagemaker prod, region:us-west-2                                  ]8;id=392834;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=795118;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

### Discover and update Finetuning options

Each of the technique and model has overridable hyperparameters that can be finetuned by the user.

In [3]:
print("Default Finetuning Options:")
pprint(rlvr_trainer.hyperparameters.to_dict()) # rename as hyperparameters

#set options
rlvr_trainer.hyperparameters.get_info()



Default Finetuning Options:


{
│   'data_path': 'None',
│   'global_batch_size': '64',
│   'learning_rate': '1e-05',
│   'max_epochs': '2',
│   'max_prompt_length': '1024',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   'name': 'example-name-3pl5q',
│   'output_path': '/opt/ml/model',
│   'preset_reward_function': '',
│   'results_directory': '',
│   'resume_from_path': '',
│   'reward_lambda_arn': '',
│   'rollout': '8',
│   'train_val_split_ratio': '0.9',
│   'validation_data_path': 'None'
}


data_path:
  Current value: None
  Type: string
  Default: None
  Required: Yes

global_batch_size:
  Current value: 64
  Type: integer
  Default: 64
  Valid options: [64, 128, 256, 512, 1024]
  Required: Yes

learning_rate:
  Current value: 1e-05
  Type: float
  Default: 1e-05
  Range: 1e-07 - 0.001
  Required: Yes

max_epochs:
  Current value: 2
  Type: integer
  Default: 2
  Range: 1 - 30
  Required: Yes

max_prompt_length:
  Current value: 1024
  Type: integer
  Default: 1024
  Range: 512 - 16384
  Required: Yes

mlflow_run_id:
  Current value: 
  Type: string
  Default: 

mlflow_tracking_uri:
  Current value: 
  Type: string
  Default: 

model_name_or_path:
  Current value: meta-llama/Llama-3.2-1B-Instruct
  Type: string
  Default: meta-llama/Llama-3.2-1B-Instruct
  Required: Yes

name:
  Current value: example-name-3pl5q
  Type: string
  Default: example-name-3pl5q
  Required: Yes

output_path:
  Current value: /opt/ml/model
  Type: string
  Default: /opt/ml/model
  Required: Ye

#### Start RLVR training


In [11]:
training_job = rlvr_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251129125943     │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          481.3s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             0.5s                      │
│   ✓    Pending           Preparing the instances for           16.9s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        10.8s                     │
│   ✓    Training          Training image download completed.    411.7s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    38.9s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [12]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251129125943',
│   training_job_arn='arn:aws:sagemaker:us-east-1:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251129125943',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://rsareddy-test-for-demo/output/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251129125943/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://rsareddy-test-for-demo/output/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251129125943/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '64',
│   │   'learning_rate': '1e-05',
│   │   'max_epochs': '2',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-3pl5q',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '8',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://rsareddy-test-for-demo/input_data/rlvr-rlaif-oss-dataset/train_285.jsonl',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://rsareddy-test-for-demo/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>,
│   │   max_pending_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x108e57a10>
│   ),
│   creation_time=datetime.datetime(2025, 11, 29, 12, 59, 

In [4]:
training_job = rlvr_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251201130759     │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          501.3s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             0.5s                      │
│   ✓    Pending           Preparing the instances for           21.4s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        10.6s                     │
│   ✓    Training          Training image download completed.    416.8s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    48.7s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251125142110")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251125142110',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251125142110',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   model_artifacts=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   training_job_output=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   training_job_status='InProgress',
│   secondary_status='Pending',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '64',
│   │   'learning_rate': '1e-05',
│   │   'max_epochs': '2',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-6bbza',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '8',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   │   max_pending_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>
│   ),
│   creation_time=datetime.datetime(2025, 11, 25, 14, 21, 11, 269000, tzinfo=tzlocal()),
│   training_start_time=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   training_end_time=<sagemaker.core.utils.utils.Unassigned object at 0x122304320>,
│   last_modified_time=datetime.datetime(2025, 11, 25, 14, 21, 11, 832000, tzinfo=tzlocal()),
│   secondary_status_transitions=[
│   │   SecondaryStatusTransition(
│   │   │   status='Starting',
│   │   │   start_time=datetime.datetime(2025, 11, 25, 14, 21, 11, 269000, tzinfo=tzlocal()),
│   │   │   end_time=datetime.datetime(2025, 11, 25, 14, 21, 11, 832000, tzinfo=tzlocal()),
│   │   │   status_message='Starting the training job'
│   │   ),
│   │   SecondaryStatusTransition(
│   │   

### View any Training job details

We can get any training job details and its status with TrainingJob.get(...)

In [6]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517',
│   processing_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   tuning_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   labeling_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   auto_ml_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123033517/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   │   'name': 'example-name-5k253',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   file_system_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   shuffle_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   keep_alive_period_in_seconds=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_reservation_ids=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_groups=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_schedules_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   training_plan_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_placement_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   warm_pool_status=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   vpc_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_i

In [14]:
training_job.refresh()
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011',
│   training_job_arn='arn:aws:sagemaker:us-east-1:052150106756:training-job/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://rsareddy-test-for-demo/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://rsareddy-test-for-demo/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124143011/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   │   'name': 'example-name-c9tcy',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://rsareddy-test-for-demo/input_data/rlvr-rlaif-oss-dataset/train_285.jsonl',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://rsareddy-test-for-demo/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x1076f5a60>,
│   │  

### Test RLVR with Custom RewardFunction

Here we are providing a user-defined reward function ARN

In [3]:

# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage] 
    model_package_group_name="sdk-test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing/input_data/rlvr-rlaif-test-data/train_285.jsonl", #Optional[]
    s3_output_path="s3://mc-flows-sdk-testing/output/",
    #sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    #role="arn:aws:iam::052150106756:role/Admin",
    custom_reward_function="arn:aws:sagemaker:us-west-2:729646638167:hub-content/sdktest/JsonDoc/rlvr-test-rf/0.0.1",
    accept_eula=True
    
)

[12/02/25 11:40:37] INFO     SageMaker session not provided. Using default Session.                  ]8;id=901012;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=326527;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

                    INFO     Runs on sagemaker prod, region:us-west-2                                  ]8;id=393681;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=548016;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

In [4]:
training_job = rlvr_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251119220405     │
│  MLFlow URL            ]8;id=684727;https://t-eq86xu3xab1s.us-west-2.experiments.sagemaker.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6IjQzOGJiZDNjLWVmZjItNDI4OS04ZDE3LWMwNTYzMmE1ZmNjMiIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNC81SnZSVjNPb1pKOUVZU3hreHBIL3J5T054ZXVFS0lsdEVtd0pkaXQyRkVBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVFclJVcHdlblF5Y2xCTWRITXlWazlNUld0elZYZE5hekI1TmpkdFQxZ3JSMjB6UjBOb1RqWjBjVkJEYjI5SFFWSnNZMlJWUTI1d2MyaElWRFI1VlVWMWR6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMWGRsYzNRdE1qbzFPVEF4T0RNM016azFNRFE2YTJWNUx6ZzNabUUxTVdReUxURTRNRGt0TkdVMFl5MWhObVV6TFRRNFpXWTNNelk1WW1NM1lnQzRBUUlCQUhndFR2ZzEyNVZtTm50WE4vTEVZV1dzREMrSk1KRzNQSUl1eU9PYTB5SU5FZ0dqY0k0dHhRVHFPQndRU3hBQU9kaWZBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU16ZHJFVGtIWUNtdGp3V1E5QWdFUWdEdGZTcTZvRXdLL3UvMGNpa1AzWE0vUUJnb1grYUFOZ2hRSUY4U2VrcTVYcGZyaFUwTGIvRmhjVEFLczFpZ09xOG0yQUg5UTVQS3RvdXdnUkFJQUFCQUFVdGxJNVdQc09pbjlXUTVhbW8vRzE3VklacU9rTkJycDlKQnFtZWNRbzVwZWpUakNmQnpJR2RrQVM3WHFvNkRqLy8vLy93QUFBQUVBQUFBQUFBQUFBQUFBQUFFQUFBUTdrRFd0TWJ5UlJ2SmFsaXFLd1creUlMNkRUbE82cXI4b01VaEZpdVVXN1JhKzQ3QWduRkM5all5Y2pRN1Y0MjU2VzdJdkYvSThnSk83Z0d1M0pqVXppeUNoSnZVajFpUUMzUGgrSkpkNzJIRFlNL2Zrd1hncjZBZlVvRTFGeGwxc1JPMXRiZ0ErVXRVemNoNldGWlVJQUtrVDZ3cHdTZ0NKZDl0Z1Z0UVhidlFBVGk5Nkd4TXhwNVpnTXQyYlV3Yi8zTUJGMkxDdWxRWWxuMVVLL3k2RndkdGVZVmc2MUNqMkJPczRkY0F4enBPbWMyQ1lpQnJacDZQcWYxWVlNcUN0TmdycVlYdjJoMDZsNFk2a0VWRWRsUHRxQlR2Y3lTTVBnVEhRMytLWjVWUitaWHZGN2s1WnJEc2h3Q283YkZ6Mk9TWHpCMHdiTzRtTjA0M1BVMnJWOU9KbGxjYmZ1WUE0eUgyd29aY0hwUUtxQkxPNXkyT3gvdmF6UGVIM0lRdVNnM1VtK3RWMWVxUjZTUzYwTGI0U3Y4bHZtdHV3ZzBSa20wMFhhYUh3V2lMVGR3dk16OURGNE9hZ012dTBDQUduR1pOVitLV2FZcnE1c05ubk1ZdGZFRlZHOHJWamVsQytVbXlQRzBlT0xZT0FKNkVCQ1JwYkRaYjl6Tmg4QTdnYXRtSkVHYnZ4Mk5KMjRZTUwwR2pBOE4wSVBVdHpDZE9wVEtKcldSaTN1YnBQOURMRkVZMWxseE9DTGRxU2U4T29TTzNhQTlPTDZVaDFoSHlQSi8ycmR4ZjFhZXdTeXk5SVMxc3NTN21nWjQyY1grZUs2MkFLeUdQckhVRDQ5V0ZLMnEyYVdhdFBNSFFDKy9VM2JVME44Z1F1SlpleVRGK2RGMHB5dmZVR2YwNXRvd0NLVjFEcGFnRGt6Y2Q3Z21taU5yVFRmb3dISXlobFFPZVpOVGJUKzR5dzRDTHE0THg0RndCU2VmekM2aFBSN2JqdGI2M1lpdHhPeFFJdDZsNG9rM0piTENURE40VlhDc2wzbDg0ZmtRQUpjZXlSbklBYk4xV1VMVlByekRLS3RXM1oyM2hiU21ibmRFeXJIekdINW1nUjU2a0t5UE00cVZCc1V2eWEyMkVNS283NjJjZjUwZloxYXpHZ3BGOCtLSmVpNlRhL0RLS1FkZzFITlZjeW9HeXlBMnE0RWNnOE81S3dubHU3YktjWlJwbWE5NnBiRFVoWG5YWlhKVUY4SHMrTkdmcENDRXFRbkFQVFM0Ylo1ai9WWWVTdG5FSDgxS1RCZHNPQXhXbjgvQmJNcDlyRmc4WnBPL1ZHODBESDVTYWcrZElWZkE1UlE1eVpMRWxEU2dQV2J6cWRtcUt0MmVyampPUnlFcUY0S1JXMW4zWGo4VTZHWElxWlROUFBPWDE2cFp0T2l0NkE5TXExVEJzRnFHMlkydUdVRTU1RTVMWGk1aDhBRHY4MFNWR1hiRzNheTNxRDRBMjdqcWQ3aE56aTRlVTF5QURCODV4NElKcTJ6UmtEblo5aVZvVGJwZ0NlN2VBc25BbkdLYUw0TTdvVmc3WkZRUko4cFB4T01yV09WSnNCOUo1OGVtUXdHTWkzMUVqWHF4RWgvbTV3WXh6Z3I2dGlYQUo0WWw5NlhWZVhoUm5CZTBsb1hqUitkdEtEQ0VzZkZIbjUzYytzVTVWcHMrNkdWMnUzNmhNK3Y3OXo5L0FMQjlia1BESXYvYlhmTUk2YVcxbnNRekxUMWc5LzYvcGN4WjFSdE5udUlsWDZvSXdYbHBnQXdkSVFVdmJRcXFremo1RmgxeFNlbUU0dmN5ZGRhK3dCdTBITGJCdTlDZEw5ZHVqV05jWHpsTy9Sd0szekhUZTBUbEt4bkxER0dhaTZid0hsSTV5SkpDczMrMTlEWEhHcVd3ZFNaTGJGbWpSS0RnK1UyZ0JuTUdVQ01RRGkyTis1aWhzcmVhMm02VlB2ei9BalVOWXF1YUxJejh6NUlIb0dVaTZ2K1BVNnc0U1lkYklQZmRRcnlMVTBMWGdDTUdTVmtSOEJFVEw1VisvSDhUMXV6akpxWEdFZ2l1SVc4TUd3bFdRbCtqWEdpUG4zaG9TTkhOU2pwK0J5VXMvM3BBPT0iLCJjaXBoZXJUZXh0IjoiQVFJQkFIZ3RUdmcxMjVWbU5udFhOL0xFWVdXc0RDK0pNSkczUElJdXlPT2EweUlORWdIZHZyY0RuRGtONDFFVlBRbW9XUEkwQUFBQW9qQ0Jud1lKS29aSWh2Y05BUWNHb0lHUk1JR09BZ0VBTUlHSUJna3Foa2lHOXcwQkJ3RXdIZ1lKWUlaSUFXVURCQUV1TUJFRURJVmZnV1VvNzFQM2tISGg0d0lCRUlCYlVxelJjVG41TXp5Ty80ZFJPRzdJRkhHSzVpbVQrWmFiMVMyR0JyVVdpMFZyTU5ubDJEMTBnSExnZVpPNHB5a01OVURVbkZSalE4cVUvTWgvR2lheHc4REtpZUoyaFVZdkd2U1Q4ajY2MGczVkcyb2RYM21KNHRaa2NnPT0iLCJzdWIiOiJhcm46YXdzOnNhZ2VtYWtlcjp1cy13ZXN0LTI6MDUyMTUwMTA2NzU2Om1sZmxvdy10cmFja2luZy1zZXJ2ZXIvbW1sdS1ldmFsLWV4cGVyaW1lbnQiLCJpYXQiOjE3NjM2MTg2NDYsImV4cCI6MTc2MzYx

In [4]:
training_job = rlvr_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251202114054     │
│                                                                                          │
│  Job Status            Completed                                                         │
│  Secondary Status      Completed                                                         │
│  Elapsed Time          595.0s                                                            │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             1.6s                      │
│   ✓    Pending           Preparing the instances for           15.2s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        10.7s                     │
│   ✓    Training          Training image download completed.    516.8s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    48.1s                     │
│   ✓    Completed         Training job completed                0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [5]:
#training_job.refresh()
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251202114054',
│   training_job_arn='arn:aws:sagemaker:us-west-2:729646638167:training-job/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251202114054',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251202114054/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://mc-flows-sdk-testing/output/meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251202114054/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '64',
│   │   'learning_rate': '1e-05',
│   │   'max_epochs': '2',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-3pl5q',
│   │   'output_path': '/opt/ml/model',
│   │   'reward_lambda_arn': 'arn:aws:lambda:us-west-2:729646638167:function:rlvr-oss-reward-function',
│   │   'rollout': '8',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   role_arn='arn:aws:iam::729646638167:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=S3DataSource(
│   │   │   │   │   s3_data_type='S3Prefix',
│   │   │   │   │   s3_uri='s3://mc-flows-sdk-testing/input_data/rlvr-rlaif-test-data/train_285.jsonl',
│   │   │   │   │   s3_data_distribution_type='FullyReplicated',
│   │   │   │   │   attribute_names=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   │   │   instance_group_names=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   │   │   model_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   │   │   hub_access_config=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>
│   │   │   │   ),
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   │   dataset_source=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://mc-flows-sdk-testing/output/',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>
│   ),
│   resource_config=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11b301ca0>,
│   │   max_pending_time_in_seconds=<sagemaker.core.utils.utils.Unass

In [ ]:

#meta-textgeneration-llama-3-2-1b-instruct-rlvr-20251113182932

## Continued Finetuning (or) Finetuning on Model Artifacts

#### Discover a ModelPackage and get its details

In [6]:
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage, ModelPackageGroup

#model_package_iter = ModelPackage.get_all(model_package_group_name="test-finetuned-models-gamma")
model_package = ModelPackage.get(model_package_name="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/61")

pprint(model_package)

ModelPackage(
│   model_package_name=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_package_group_name='test-finetuned-models-gamma',
│   model_package_version=61,
│   model_package_registration_type='Logged',
│   model_package_arn='arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/61',
│   model_package_description=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   creation_time=datetime.datetime(2025, 11, 23, 3, 44, 26, tzinfo=tzlocal()),
│   inference_specification=InferenceSpecification(
│   │   containers=[
│   │   │   ContainersItem(
│   │   │   │   model_data_url=None,
│   │   │   │   image=None,
│   │   │   │   nearest_model_name=None,
│   │   │   │   model_data_source=ModelDataSource(),
│   │   │   │   is_checkpoint=False,
│   │   │   │   base_model=BaseModelInfo(hub_content_name='meta-textgeneration-llama-3-2-3b-instruct')
│   │   │   )
│   │   ]
│   ),
│   source_algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   validation_specification=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_package_status='Completed',
│   model_package_status_details=ModelPackageStatusDetails(validation_statuses=[], image_scan_statuses=[]),
│   certify_for_marketplace=False,
│   model_approval_status=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   created_by=UserContext(
│   │   user_profile_arn=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   │   user_profile_name=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   │   domain_id=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   │   iam_identity=IamIdentity(
│   │   │   arn='arn:aws:sts::052150106756:assumed-role/Admin/SageMaker',
│   │   │   principal_id='AROAQYJDDPKCM5Q7HXZYK:SageMaker',
│   │   │   source_identity=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>
│   │   )
│   ),
│   metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_metrics=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   deployment_specification=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   last_modified_time=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   last_modified_by=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   approval_description=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   domain=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   task=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   sample_payload_url=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   sample_payload_content_type=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   customer_metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   drift_check_baselines=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   additional_inference_specifications=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   skip_model_validation=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   source_uri=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   security_config=ModelPackageSecurityConfig(
│   │   kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>
│   ),
│   model_card=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>,
│   model_life_cycle=<sagemaker.core.utils.utils.Unassigned object at 0x10acbc320>
)

#### Create Trainer

Trainer creation is same as above Finetuning Section except for `model`'s input is ModelPackage(previously trained artifacts)

In [7]:
# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model=model_package, # Union[str, ModelPackage] 
    training_type=TrainingType.LORA, 
    model_package_group_name="test-finetuned-models-gamma", #"test-finetuned-models", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2",     #"arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    role="arn:aws:iam::052150106756:role/Admin"
)

In [2]:
# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/61", # Union[str, ModelPackage] 
    training_type=TrainingType.LORA, 
    model_package_group_name="test-finetuned-models-gamma", #"test-finetuned-models", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2",     #"arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/MarketingDemoDataset1/1.0.0", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    #role="arn:aws:iam::052150106756:role/service-role/AmazonSageMaker-ExecutionRole-20250731T162975"
    role="arn:aws:iam::052150106756:role/Admin"
)

[11/26/25 00:17:58] INFO     Runs on sagemaker gamma, region:us-west-2                                 ]8;id=731005;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=65071;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # For fine-tuning                                                                           │
│ ❱  2 rlvr_trainer = RLVRTrainer(                                                                 │
│    3 │   model="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-    │
│    4 │   training_type=TrainingType.LORA,                                                        │
│    5 │   model_package_group_name="test-finetuned-models-gamma", #"test-finetuned-models", #     │
│                                                                                                  │
│ /Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train │
│ /rlvr_trainer.py:165 in __init__                                                                 │
│                                                                                                  │
│   162 │   │   self.networking = networking                                                       │
│   163 │   │                                                                                      │
│   164 │   │   # Initialize fine-tuning options with beta session fallback                        │
│ ❱ 165 │   │   self.hyperparameters, self._model_arn = _get_fine_tuning_options_and_model_arn(s   │
│   166 │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │    CustomizationTechni   │
│   167 │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │    self.sagemaker_sess   │
│   168 │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │   │    sagemaker_session=s   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: cannot unpack non-iterable NoneType object

#### Start the Training

In [ ]:
training_job = rlvr_trainer.train(
    wait=True,
)

[11/24/25 14:01:03] INFO     Training Job Name:                                                 ]8;id=355721;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py\rlvr_trainer.py]8;;\:]8;id=369327;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py#111\111]8;;\
                             meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124140103                         

Training Job Name: meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251124140103


                    INFO     MLflow resource ARN:                                             ]8;id=100728;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=480551;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#436\436]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

                    INFO     Creating training_job resource.                                     ]8;id=436922;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=176368;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

In [11]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854',
│   processing_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   tuning_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   labeling_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   auto_ml_job_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-3b-instruct-rlvr-20251123035854/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   hyper_parameters={
│   │   'data_path': 'None',
│   │   'global_batch_size': '256',
│   │   'learning_rate': '3e-05',
│   │   'lora_alpha': '32',
│   │   'max_epochs': '1',
│   │   'max_prompt_length': '1024',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-3B-Instruct',
│   │   'name': 'example-name-5k253',
│   │   'output_path': '/opt/ml/model',
│   │   'rollout': '6',
│   │   'train_val_split_ratio': '0.9',
│   │   'validation_data_path': 'None'
│   },
│   algorithm_specification=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   file_system_data_source=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/rlvr-rlaif-test-dataset/0.0.2'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   shuffle_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   keep_alive_period_in_seconds=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_reservation_ids=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_groups=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   capacity_schedules_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   training_plan_arn=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   │   instance_placement_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>
│   ),
│   warm_pool_status=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   vpc_config=<sagemaker_core.main.utils.Unassigned object at 0x1141f1390>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_i

#### Nova RLVR job

In [2]:
import os
os.environ['SAGEMAKER_REGION'] = 'us-east-1'

# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="nova-textgeneration-lite-v2", # Union[str, ModelPackage] 
    model_package_group_name="sdk-test-finetuned-models", #"test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-east-1:052150106756:mlflow-app/app-UNBKLOAX64PX",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-nova-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-nova-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="s3://mc-flows-sdk-testing-us-east-1/input_data/rlvr-nova/grpo-64-sample.jsonl",
    validation_dataset="s3://mc-flows-sdk-testing-us-east-1/input_data/rlvr-nova/grpo-64-sample.jsonl",
    s3_output_path="s3://mc-flows-sdk-testing-us-east-1/output/",
    custom_reward_function="arn:aws:sagemaker:us-east-1:729646638167:hub-content/sdktest/JsonDoc/rlvr-nova-test-rf/0.0.1",
    accept_eula=True
)


[12/02/25 17:54:25] INFO     SageMaker session not provided. Using default Session.                  ]8;id=1564;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=880984;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

                    INFO     Runs on sagemaker prod, region:us-east-1                                  ]8;id=749936;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=798304;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

[{'DisplayName': 'Nova Lite V2 LoRA RLVR SMTJ training on GPU', 'Name': 'nova_lite_v2_smtj_p5_p5en_gpu_lora_rft', 'RecipeFilePath': 'recipes/fine-tuning/nova/nova_2_0/nova_lite/RFT/nova_lite_v2_smtj_p5_p5en_gpu_lora_rft.yaml', 'CustomizationTechnique': 'RLVR', 'InstanceCount': 4, 'Type': 'FineTuning', 'Versions': ['1.0'], 'Hardware': 'GPU', 'SupportedInstanceTypes': ['ml.p5.48xlarge', 'ml.p5en.48xlarge'], 'Peft': 'LORA', 'SequenceLength': '8K', 'ServerlessMeteringType': 'Hourly', 'SmtjRecipeTemplateS3Uri': 's3://jumpstart-cache-prod-us-east-1/recipes/nova_lite_v2_smtj_p5_p5en_gpu_lora_rft_payload_template_sm_jobs_v1.0.19.yaml', 'SmtjOverrideParamsS3Uri': 's3://jumpstart-cache-prod-us-east-1/recipes/nova_lite_v2_smtj_p5_p5en_gpu_lora_rft_override_params_sm_jobs_v1.0.19.json', 'SmtjImageUri': '708977205387.dkr.ecr.us-east-1.amazonaws.com/nova-fine-tune-repo:SM-TJ-RFT-V2-latest'}, {'DisplayName': 'Nova Lite V2 Full Rank RLVR SMTJ training on GPU', 'Name': 'nova_lite_v2_smtj_p5_p5en_gpu_rf

In [3]:
rlvr_trainer.hyperparameters.to_dict()

{'name': 'my-rft-run-dcx1l',
 'data_s3_path': 's3://<bucket>/<data file>',
 'reward_lambda_arn': 'arn:aws:lambda:<region>:<account-id>:function:<function-name>',
 'learning_rate': '0.0001',
 'max_steps': '10',
 'lora_alpha': '32',
 'global_batch_size': '64',
 'max_length': '8192',
 'learning_rate_ratio': '64.0'}

In [4]:
rlvr_trainer.hyperparameters.data_s3_path = 's3://example-bucket'

rlvr_trainer.hyperparameters.reward_lambda_arn = 'arn:aws:lambda:us-east-1:729646638167:function:rlvr-nova-reward-function'

In [5]:
rlvr_trainer.hyperparameters.to_dict()

{'name': 'my-rft-run-dcx1l',
 'data_s3_path': 's3://example-bucket',
 'reward_lambda_arn': 'arn:aws:lambda:us-east-1:729646638167:function:rlvr-nova-reward-function',
 'learning_rate': '0.0001',
 'max_steps': '10',
 'lora_alpha': '32',
 'global_batch_size': '64',
 'max_length': '8192',
 'learning_rate_ratio': '64.0'}

In [6]:
training_job = rlvr_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      nova-textgeneration-lite-v2-rlvr-20251202164326                   │
│                                                                                          │
│  Job Status            Failed                                                            │
│  Secondary Status      Failed                                                            │
│  Elapsed Time          643.7s                                                            │
│  Failure Reason        AlgorithmError: Algorithm container exited with error. Please     │
│                        try again.                                                        │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             1.4s                      │
│   ✓    Pending           Preparing the instances for           35.1s                     │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        451.3s                    │
│   ✓    Training          Training image download completed.    136.3s                    │
│                          Training in progress.                                           │
│   ✓    Uploading         Uploading generated training model    18.9s                     │
│   ✓    Failed            Training job failed                   0.0s                      │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
training_job = rlvr_trainer.train(wait=False)

[12/02/25 17:54:47] INFO     SageMaker session not provided. Using default Session.                  ]8;id=771974;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=156715;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

[12/02/25 17:54:48] INFO     Role not provided. Using default role:                                  ]8;id=691076;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=299194;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#75\75]8;;\
                             arn:aws:iam::729646638167:role/Admin                                                  

                    INFO     Training Job Name: nova-textgeneration-lite-v2-rlvr-20251202175448 ]8;id=20384;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py\rlvr_trainer.py]8;;\:]8;id=748925;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/rlvr_trainer.py#182\182]8;;\

[12/02/25 17:54:49] INFO     Using first available ready MLflow app:                           ]8;id=531321;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=986206;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#74\74]8;;\
                             arn:aws:sagemaker:us-east-1:729646638167:mlflow-app/app-J2NPD6IV7                     
                             7BJ                                                                                   

                    INFO     MLflow resource ARN:                                             ]8;id=915976;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=393836;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#466\466]8;;\
                             arn:aws:sagemaker:us-east-1:729646638167:mlflow-app/app-J2NPD6IV                      
                             77BJ                                                                                  

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


                    INFO     Creating training_job resource.                                     ]8;id=414140;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=554749;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

#### Nova RLVR job (551952248621)

In [3]:
import os
os.environ['SAGEMAKER_REGION'] = 'us-east-1'

# For fine-tuning 
rlvr_trainer = RLVRTrainer(
    model="nova-textgeneration-lite-v2", # Union[str, ModelPackage] 
    model_package_group_name="test-prod-iad-model-pkg-group", #"test-finetuned-models", # Make it Optional
    #mlflow_resource_arn="arn:aws:sagemaker:us-east-1:052150106756:mlflow-app/app-UNBKLOAX64PX",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-nova-rlvr-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-nova-rlvr-finetuned-models-run", # Optional[str]
    training_dataset="s3://ease-integ-test-input-551952248621-us-east-1/converse-serverless-test-data/grpo-64-sample.jsonl",
    validation_dataset="s3://ease-integ-test-input-551952248621-us-east-1/converse-serverless-test-data/grpo-64-sample.jsonl",
    s3_output_path="s3://ease-integ-test-output-551952248621-us-east-1/model-customization-algo/",
    custom_reward_function="arn:aws:sagemaker:us-east-1:551952248621:hub-content/recipestest/JsonDoc/nova-prod-iad-test-evaluator-lambda-reward-function/0.0.1",
    accept_eula=True
)


[12/02/25 17:03:19] INFO     SageMaker session not provided. Using default Session.                  ]8;id=689545;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py\defaults.py]8;;\:]8;id=727438;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/defaults.py#61\61]8;;\

                    INFO     Runs on sagemaker prod, region:us-east-1                                  ]8;id=246524;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py\utils.py]8;;\:]8;id=507523;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/utils/utils.py#354\354]8;;\

[{'DisplayName': 'Nova Lite V2 LoRA RLVR SMTJ training on GPU', 'Name': 'nova_lite_v2_smtj_p5_p5en_gpu_lora_rft', 'RecipeFilePath': 'recipes/fine-tuning/nova/nova_2_0/nova_lite/RFT/nova_lite_v2_smtj_p5_p5en_gpu_lora_rft.yaml', 'CustomizationTechnique': 'RLVR', 'InstanceCount': 4, 'Type': 'FineTuning', 'Versions': ['1.0'], 'Hardware': 'GPU', 'SupportedInstanceTypes': ['ml.p5.48xlarge', 'ml.p5en.48xlarge'], 'Peft': 'LORA', 'SequenceLength': '8K', 'ServerlessMeteringType': 'Hourly', 'SmtjRecipeTemplateS3Uri': 's3://jumpstart-cache-prod-us-east-1/recipes/nova_lite_v2_smtj_p5_p5en_gpu_lora_rft_payload_template_sm_jobs_v1.0.19.yaml', 'SmtjOverrideParamsS3Uri': 's3://jumpstart-cache-prod-us-east-1/recipes/nova_lite_v2_smtj_p5_p5en_gpu_lora_rft_override_params_sm_jobs_v1.0.19.json', 'SmtjImageUri': '708977205387.dkr.ecr.us-east-1.amazonaws.com/nova-fine-tune-repo:SM-TJ-RFT-V2-latest'}, {'DisplayName': 'Nova Lite V2 Full Rank RLVR SMTJ training on GPU', 'Name': 'nova_lite_v2_smtj_p5_p5en_gpu_rf

In [4]:
rlvr_trainer.hyperparameters.to_dict()

{'name': 'my-rft-run-dcx1l',
 'data_s3_path': 's3://<bucket>/<data file>',
 'reward_lambda_arn': 'arn:aws:lambda:<region>:<account-id>:function:<function-name>',
 'learning_rate': '0.0001',
 'max_steps': '10',
 'lora_alpha': '32',
 'global_batch_size': '64',
 'max_length': '8192',
 'learning_rate_ratio': '64.0'}

In [5]:
rlvr_trainer.hyperparameters.data_s3_path = 's3://example-bucket'

rlvr_trainer.hyperparameters.reward_lambda_arn = 'arn:aws:lambda:us-east-1:729646638167:function:rlvr-nova-reward-function'

In [6]:
rlvr_trainer.hyperparameters.to_dict()

{'name': 'my-rft-run-dcx1l',
 'data_s3_path': 's3://example-bucket',
 'reward_lambda_arn': 'arn:aws:lambda:us-east-1:729646638167:function:rlvr-nova-reward-function',
 'learning_rate': '0.0001',
 'max_steps': '10',
 'lora_alpha': '32',
 'global_batch_size': '64',
 'max_length': '8192',
 'learning_rate_ratio': '64.0'}

In [ ]:
training_job = rlvr_trainer.train(wait=True)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      nova-textgeneration-lite-v2-rlvr-20251202170403                   │
│                                                                                          │
│  Job Status            InProgress                                                        │
│  Secondary Status      Training                                                          │
│  Elapsed Time          2307.0s                                                           │
│                                                                                          │
│ Status Transitions                                                                       │
│                                                                                          │
│        Step              Details                               Duration                  │
│  ───────────────────────────────────────────────────────────────────────────             │
│   ✓    Starting          Starting the training job             1.4s                      │
│   ✓    Pending           Preparing the instances for           110.8s                    │
│                          training                                                        │
│   ✓    Downloading       Downloading the training image        441.1s                    │
│        Training          [████████████████████] 100.0%                                   │
│                          - Epoch 10/10, Step 1/1                                         │
│                                                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────╯